# Elasticsearch queries

Repasemos las posibles queries que ofrece elastic, desde las mas comunes hasta algunas mas complejas.  

## Setup

Lo primero es declarar el cliente y un par de metodos para facilitar los ejemplos.

In [ ]:
from elasticsearch import Elasticsearch
from dateutil.parser import parse as parse_date


es = Elasticsearch("http://elasticsearch:9200")
es.info()

def print_hits(results):
    " Simple utility function to print results of a search query. "
    print_search_stats(results)
    for hit in results['hits']['hits']:
        # get created date for a repo and fallback to authored_date for a commit
        print('/%s/%s/%s [%s]: |%s| %s - %s' % (
                hit['_index'], hit['_type'], hit['_id'], hit['_score'],
                hit['_source']['play_name'].split('\n')[0],
                hit['_source']['speaker'].split('\n')[0],
                hit['_source']['text_entry']))

    print('=' * 80)
def print_search_stats(results):
    print('=' * 80)
    print('Total %d found in %dms' % (results['hits']['total'], results['took']))
    print('-' * 80)

def search_query(query):
    """Executes a call to elastic via q param"""
    print_hits(es.search(index='shakespeare', params={"q": query}))


def search_query_body(body):
    """Executes a call to elastic via q param"""
    print_hits(es.search(index='shakespeare', body=body))

## Busquedas

### Buscar en el campo `_all`

Elastic permite hacer una busqueda en todos los campos a la vez, gracias a  
un campo especial llamado `_all` disponible en las queries:

In [ ]:
search_query("love")

### Busqueda por campos

Tambien podemos hacer busquedas directas en cada campo:

In [ ]:
search_query("speaker:(ROMEO OR JULIET)")

In [ ]:
search_query("speaker:(ROMEO OR JULIET) AND NOT speaker:JULIET")

### Busqueda con boosts

Tambien podemos hacer busquedas con ciertos campos potenciados:

In [ ]:
search_query("text_entry:love AND (speaker:ROMEO^5 OR speaker:JULIET)")

### Busqueda con comodines o wildcards

Tambien podemos hacer busquedas con comodines dentro de campos:

In [ ]:
search_query("play_name:ot?ello")

In [ ]:
search_query("play_name:K* AND NOT play_name:*John*")

In [ ]:
search_query("text_entry:kil? AND speaker:(*king*^4)")

### Busqueda con Fuzziness

Tambien podemos hacer busquedas con fuzzyness:

In [ ]:
search_query("text_entry:inocent~1")

In [ ]:
search_query_body({
  "query": {
    "query_string": {
      "query": "love AND (NOT play_name:\"Romeo and Juliet\")"
    }
  }
})

## Indexes y maps

Antes de insertar datos es recomendable siempre crear el indice y sus mapeos:

In [ ]:
es.indices.create(index='documents_february', body={
  "settings": {
    "number_of_replicas": 1,
    "number_of_shards": 3,
    "analysis": {},
    "refresh_interval": "1s"
  },
  "mappings": {
    "document": {
      "properties": {
        "title": {
          "type": "text",
          "analyzer": "english"
        }
      }
    }
  }
})
# DEPRECATED! 1 type in elastic > 6.0.0

Esto es equivalente a:
```
PUT /documents
{
  "settings": {
    "number_of_replicas": 1,
    "number_of_shards": 3,
    "analysis": {},
    "refresh_interval": "1s"
  },
  "mappings": {
    "title_text": {
      "properties": {
        "title": {
          "type": "text",
          "analyzer": "english"
        }
      }
    }
  }
}
```

In [ ]:
es.indices.get("documents_february")
# Equivalente a GET /documents_february [_settings|_mappings]

### Modificar tipos

In [ ]:
es.indices.put_mapping(index='documents_february', doc_type='document', body={
  "document": {
    "properties": {
      "content": {
          "type": "text",
          "analyzer": "english"
      },
    }
  }
})
es.indices.put_mapping(index='documents_february', doc_type='document', body={
  "document": {
    "properties": {
      "tag": {
          "type": "keyword"
      },
    }
  }
})
es.indices.get("documents_february")

Esto es equivalente a:
    ```
    PUT /documents_february/_mapping/document
    {
      "document": {
        "properties": {
          "tag": {
            "type": "keyword"
          }
        }
      }
    }
    ```
    
## Manejo de documentos

Hagamos CRUD sobre documentos:

In [ ]:
es.create(index='documents_february', doc_type='document', id=0, body={
    "title": "New Document",
    "content": "This is a new document for the master class",
    "tag": ["general", "testing"]
})

Equivalente a:
```
PUT /documents_february/document/1
{
  "title": "New Document",
  "content": "This is a new document for the master class",
  "tag": [
    "testing"
  ]
}
-----

POST /documents_february/document
{
  "title": "New Document",
  "content": "This is a new document for the master class",
  "tag": [
    "testing"
  ]
}
```

### Lectura

In [ ]:
es.get(index='documents_february', doc_type='document', id=0)

Equivalente a:
```
GET /documents_february/document/0
```

### Borrado

In [ ]:
es.delete(index='documents_february', doc_type='document', id=0)

Equivalente a:
```
DELETE /documents_february/document/0
```

## Agregacion y Queries complejas

Elastic permite, gracias a su API de agregacion, realizar queries mas complejas:

```
POST /_search

{
    "size":0,
    "aggs" : {
        "Popular plays" : {
            "terms" : {
                "field" : "play_name.keyword"
            }
        }
    }
}

-------
{
    "size":0,
    "aggs" : {
        "Total plays" : {
            "terms" : {
                "field" : "play_name.keyword"
            },
            "aggs" : {
             "Per type" : {
                 "terms" : {
                     "field" : "speaker.keyword"
                  }
             }
            }
        }
    }
}

-------
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "speaker": "*king*"
          }
        }
      ]
    }
  },
  "aggs": {
    "my_agg": {
      "terms": {
        "field": "play_name.keyword",
        "size": 10
      }
    }
  },
  "sort": [
    {
      "play_name.keyword": {
        "order": "desc"
      }
    }
  ]
}
```